## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![图片描述](hw4_img/hw4_01.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：
首先借助AI回顾了一下定义，最小生成树：所有生成树中，边权和最小。瓶颈生成树：所有生成树中，最大边权最小  
要证：MST 的最大边权等于所有生成树中最大边权的最小值，即也是 BST  
证明思路：
设 $T$ 是图 $G$ 的一棵最小生成树（MST），我们要证明 $T$ 同时也是一棵 BST  
反证法：  
假设 $T$ 不是一棵瓶颈生成树。
则存在另一棵生成树 $T'$，使得：  
$\max_{e \in T'} w(e) < \max_{e \in T} w(e)$  
即 $T'$ 的瓶颈值小于 $T$ 的瓶颈值。
设 $w_{\max}(T) = w(e_{\text{max}})$，其中 $e_{\text{max}} \in T$ 是 T 中的最大权重边。
我们考虑从 $T'$ 中移除任意一个 $e' \in T'$，添加 $e_{\text{max}}$（若其连接的两个点在 $T'$ 中不连通），尝试构造出一棵新的生成树。这将导致边权和减小或不变，说明 $T'$ 的权重和不大于 $T$。
但这与 $T$ 是最小生成树矛盾。
因此，MST 中的最大边权已经是所有生成树中最小的最大边权，所以它一定也是 BST  
第二题：  
核心思路是：只保留权重 ≤ b 的边，判断图是否连通。  
如果保留的子图仍连通（也就是说包含一棵生成树），那么就存在一棵瓶颈值 ≤ b 的生成树。  
否则，不存在  
仅访问所有边和点一次，复杂度为 $O(|V| + |E|)$，即线性时间  
第三题：  
思路：二分 + 判断子图是否连通
二分边权值范围 $[w_{\min}, w_{\max}]$：  
每次猜一个瓶颈值 $b$。  
用上面的方法判断图中是否存在一棵生成树最大边权 ≤ b。  
找到最小可行瓶颈值 $b^*$。  
用 $b^*$ 构建子图，找任意一棵生成树（如用 DFS + Union-Find）  
时间复杂度：  
二分查找次数为 $O(\log W)$，其中 $W$ 为最大边权。

每次判断图连通性为 $O(V+E)$。

构建生成树也为 $O(V+E)$。

总体复杂度为 线性对数时间


In [1]:
# add your code here
from collections import defaultdict, deque

def exists_bst_with_bottleneck(G_edges, n, b):
    # G_edges: list of (u, v, w)
    # n: number of vertices
    # b: candidate bottleneck value
    
    graph = defaultdict(list)
    for u, v, w in G_edges:
        if w <= b:
            graph[u].append(v)
            graph[v].append(u)

    # BFS to check connectivity
    visited = set()
    queue = deque([0])
    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            queue.extend(graph[node])

    return len(visited) == n
# algorithm of the liear time complexity 

In [2]:
def find_bottleneck_spanning_tree(G_edges, n):
    weights = sorted(set(w for _, _, w in G_edges))
    
    def is_connected(b):
        graph = defaultdict(list)
        for u, v, w in G_edges:
            if w <= b:
                graph[u].append(v)
                graph[v].append(u)
        visited = set()
        queue = deque([0])
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                queue.extend(graph[node])
        return len(visited) == n

    # Binary search to find minimum b
    left, right = 0, len(weights) - 1
    res_b = weights[-1]
    while left <= right:
        mid = (left + right) // 2
        if is_connected(weights[mid]):
            res_b = weights[mid]
            right = mid - 1
        else:
            left = mid + 1

    # Construct tree with edges <= res_b
    parent = list(range(n))
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(x, y):
        px, py = find(x), find(y)
        if px == py:
            return False
        parent[px] = py
        return True

    tree = []
    for u, v, w in sorted(G_edges, key=lambda x: x[2]):
        if w <= res_b and union(u, v):
            tree.append((u, v, w))
        if len(tree) == n - 1:
            break

    return tree, res_b

In [3]:
edges = [(0, 1, 4), (0, 2, 3), (1, 2, 5), (1, 3, 6), (2, 3, 2)]
n = 4
tree, bottleneck = find_bottleneck_spanning_tree(edges, n)
print("瓶颈生成树：", tree)
print("瓶颈值：", bottleneck)

瓶颈生成树： [(2, 3, 2), (0, 2, 3), (0, 1, 4)]
瓶颈值： 4


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


这是一个 图优化问题，目标是从候选边集合 $E'$ 中选择一条新建道路，使得在原图 $G = (V, E, l)$ 的基础上添加这条边后，使两个给定城市 $s$ 和 $t$ 之间的最短路径减少最多  
算法设计：基于 Dijkstra + 遍历候选边  
时间复杂度:  
Dijkstra 两次：$O((|V| + |E|) \log |V|)$。  
遍历 $E'$：$O(|E'|)$。  
总复杂度为  
$O((|V| + |E|) \log |V| + |E'|)$  


In [4]:
import heapq
from collections import defaultdict

def dijkstra(n, graph, start):
    dist = [float('inf')] * n
    dist[start] = 0
    heap = [(0, start)]
    while heap:
        d, u = heapq.heappop(heap)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if dist[v] > d + w:
                dist[v] = d + w
                heapq.heappush(heap, (dist[v], v))
    return dist

def best_road_to_add(n, E, E_new, l_new, s, t):
    # Step 1: Build graph and reverse graph
    graph = defaultdict(list)
    reverse_graph = defaultdict(list)
    for u, v, w in E:
        graph[u].append((v, w))
        reverse_graph[v].append((u, w))
    
    # Step 2: Run Dijkstra from s and t
    dist_s = dijkstra(n, graph, s)
    dist_t = dijkstra(n, reverse_graph, t)
    original_dist = dist_s[t]
    
    # Step 3: Try all candidate new edges
    max_improvement = 0
    best_edge = None
    for (u, v), w in zip(E_new, l_new):
        if dist_s[u] != float('inf') and dist_t[v] != float('inf'):
            new_len = dist_s[u] + w + dist_t[v]
            improvement = original_dist - new_len
            if improvement > max_improvement:
                max_improvement = improvement
                best_edge = (u, v)
    
    return best_edge, max_improvement


In [5]:
n = 5
E = [(0,1,2), (1,2,3), (2,4,2), (0,3,10), (3,4,1)]
E_new = [(1,3), (3,2), (0,4)]
l_new = [1, 2, 7]
s, t = 0, 4

best_edge, max_delta = best_road_to_add(n, E, E_new, l_new, s, t)
print(f"最优新建道路: {best_edge}, 路程减少: {max_delta}")


最优新建道路: (1, 3), 路程减少: 3


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：  
这道题没什么思路，甚至题目都都不太懂，是借助AI理解的题意，但是怎么做还是不明白，借助AI大致理解了怎么做  
第一题：  
为了将一个具有顶点和边容量的网络转换为仅具有边容量的网络，我们可以使用顶点拆分技术。这种方法的基本思想是将每个具有容量限制的顶点拆分成多个顶点，并通过边连接这些顶点，以模拟原始顶点的容量限制  
步骤:  
![图片描述](hw4_img/hw4_4.1.png)  
![图片描述](hw4_img/hw4_4.2.png)  
![图片描述](hw4_img/hw4_4.3.png)  
![图片描述](hw4_img/hw4_4.4.png)  